In [2]:
using TensorDec
using DynamicPolynomials
using MomentTools
using CSDP, JuMP
# The function "Optimizer" is a global optimization solver based on positive semi-definite programming
optimizer = CSDP.Optimizer
using LinearAlgebra

# Example 1:

In [3]:
# Define the parameters
X = @polyvar x1 x2 x3

(x1, x2, x3)

In [4]:
# P is a homogeneous polynomial of degree 4 in 3 variables
P = (x1+x2+0.75*x3)^4+1.5*(x1-x2)^4-2*(x1-x3)^4;

The graph of P in polar coordinates on the sphere looks like this:

![title](pol_img.png)

Let us use the function "optimizer" to minimize and maximize P on the unit sphere. The maximum evaluation of P <b>in absolute value</b> on the unit sphere (and that is why we have to use both maximize and minimize functions) gives the spectral norm of P and equivalently a best rank-1 approximation of the symmetric tensor associated to P.

In [5]:
v1, M1 = minimize(P, [x1^2+x2^2+x3^2-1], [], X, 8, optimizer);
v2, M2 = maximize(P, [x1^2+x2^2+x3^2-1], [], X, 8, optimizer);

CSDP 6.2.0
Iter:  0 Ap: 0.00e+000 Pobj:  0.0000000e+000 Ad: 0.00e+000 Dobj:  0.0000000e+000 
Iter:  1 Ap: 6.21e-001 Pobj: -4.7686654e+001 Ad: 3.23e-001 Dobj:  7.7359277e+000 
Iter:  2 Ap: 1.00e+000 Pobj: -6.7310911e+002 Ad: 3.23e-001 Dobj:  9.7555745e+000 
Iter:  3 Ap: 1.00e+000 Pobj: -6.2816076e+002 Ad: 8.92e-001 Dobj:  1.2402191e+000 
Iter:  4 Ap: 1.00e+000 Pobj: -3.7367096e+002 Ad: 8.58e-001 Dobj:  7.2516440e-001 
Iter:  5 Ap: 1.00e+000 Pobj: -1.6533594e+002 Ad: 7.86e-001 Dobj:  7.3251215e-001 
Iter:  6 Ap: 1.00e+000 Pobj: -5.4922060e+001 Ad: 8.87e-001 Dobj:  6.4477629e-001 
Iter:  7 Ap: 9.46e-001 Pobj: -1.3326049e+001 Ad: 8.77e-001 Dobj:  1.9081239e-001 
Iter:  8 Ap: 6.21e-001 Pobj: -8.0638501e+000 Ad: 8.43e-001 Dobj: -2.7976807e+000 
Iter:  9 Ap: 1.00e+000 Pobj: -8.9630052e+000 Ad: 5.89e-001 Dobj: -6.4430198e+000 
Iter: 10 Ap: 9.08e-001 Pobj: -7.8083918e+000 Ad: 9.34e-001 Dobj: -7.5587006e+000 
Iter: 11 Ap: 8.99e-001 Pobj: -7.7124422e+000 Ad: 8.30e-001 Dobj: -7.6744224e+000 
Iter:

The minimum evaluation of P on the unit sphere is: -7.701579459519532.

The maximum evaluation of P on the unit sphere is: 6.565249952183416.

Thus, the maximum weight in absolute value which is the spectral norm of P is: 7.701579459519532.

The unit vectors that give this value are: [0.6805571886747267, 0.048429787707634814, -0.7310926539221407] and [-0.6805571886747267, -0.048429787707634814, 0.7310926539221407].

Let us compute a rank-1 approximation of P. We will compute an initial point by the method SMD, the Julia function that corresponds to this method in the package TensorDec is called "decompose", then we will use the Riemannian Newton algorithm with trust region scheme for the real case, the corresponding Julia function in the package TensorDec is called "rne_n_tr_r".

In [16]:
# Compute an initial point
w1, V1 = decompose(P,1)

([-7.089690728998527], [0.6841218273996007; 0.04220544010380849; -0.7281456077606144], Dict{String,Any}("diagonalization" => Dict{String,Any}("case" => "1x1")))

We can notice that the weight given by decompose is very close to the one given by Optimizer. Let us refine this point by using a few number of iterations of rne_n_tr_r, for example 5 iterations.

In [26]:
w_end, V_end = rne_n_tr_r(P, w1, V1, Dict{String,Any}("maxIter" => 5,"epsIter" => 1.e-3))

([-7.701576525649196], [0.68061769889553; 0.04831896554028091; -0.7310436550024019], Dict{String,Any}("d*" => 8.819736152809341,"d0" => 8.841950556430849,"nIter" => 5,"epsIter" => 0.001,"maxIter" => 5))

The weight in absolute value given by rne_n_tr_r initialized by decompose for rank-1 symmetric tensor approximation is: 7.701576525649196.

The unit vector given by rne_n_tr_r initialized by decompose for rank-1 symmetric tensor approximation is: [0.68061769889553; 0.04831896554028091; -0.7310436550024019].

Verifying with the global optimization method "optimizer" from the package CSDP, the symmetric rank-1 approximation $w_{end}(v_{end}^tX)^4$ of P given by "rne_n_tr_r" initialized by "decompose" is a best rank-1 approximation.

# Example 2:

Let us take a random symmetric tensor normally distributed with complex coefficients of order 4 and dimension 3 (the generic symmetric rank is 5), and let us compute by the Riemannian Newton algorithm "rne_n_tr" and the Riemannian Gauss--Newton algorithm "rgn_v_tr" initialized by a random initial point obeying normal distribution an approximated rank-3 symmetric tensor.   

In [38]:
# Take a random symmetric tensor
using Tensors
n = 3; d = 4; r = 3
T = randn(SymmetricTensor{d, n})+randn(SymmetricTensor{d, n})*im
T = convert(Array,T)
# show the first 3 arrays of T
T[:,:,:,1]

3×3×3 Array{Complex{Float64},3}:
[:, :, 1] =
    0.304489-2.14852im   -0.00230603+2.81im      -0.621412-0.185586im
 -0.00230603+2.81im          1.26159-0.687682im   0.677236-0.314596im
   -0.621412-0.185586im     0.677236-0.314596im   0.652919-1.50752im

[:, :, 2] =
  0.114497+0.472617im  -0.454587+2.23814im    -0.215587-1.08674im
 -0.454587+2.23814im    0.765382+0.970151im      2.2932-1.98527im
 -0.215587-1.08674im      2.2932-1.98527im   -0.0570214+0.405164im

[:, :, 3] =
 -1.38547-0.945676im   0.319933+0.53044im   0.554848+1.60772im
 0.319933+0.53044im   -0.323876+0.698715im  -1.67809-2.0286im
 0.554848+1.60772im    -1.67809-2.0286im    -1.38897+0.225804im

In [39]:
# Take the associate homogeneous polynomial P to T by applying the function ahp (for associate homogeneous polynomial)
X = (@polyvar x[1:n])[1]
P = ahp(T, X);

In [58]:
# Take an initial point 
w = ones(r) + fill(0.0+0.0im,r);
V = randn(ComplexF64,n,r);

# Apply rne_n_tr
w_end, V_end, Info = rne_n_tr(P, w, V, Dict{String,Any}("maxIter" => 500, "epsIter" => 1.e-3))

([3.3533378582723405, 1.7528646663899954, 5.021876225230323], Complex{Float64}[-0.610048013088563 + 0.2278059897255048im 0.7944923164400758 + 0.034592585203756895im -0.04277847243642945 - 0.4515491977610789im; 0.5078071901018043 + 0.4498236972987523im 0.3999428679776847 + 0.36458354731448234im 0.4336335307863758 - 0.3802580876158061im; -0.3373082465526216 + 0.044266224244519994im -0.26111415928223225 + 0.08080375839222821im 0.6788726483700402 - 0.027766875461433867im], Dict{String,Any}("d*" => 7.856359646235264,"d0" => 31.118041861573218,"nIter" => 10,"epsIter" => 0.001,"maxIter" => 500))

In [59]:
# Adjust the initial point to use't with rgn_v_tr since this function take only the matrix V as parameters without the weight vector
for i in 1:r
    V[:,i]=(w[i])^(1/d)*V[:,i]
end

# Apply rgn_v_tr
V_end, Info = rgn_v_tr(P, V, Dict{String,Any}("maxIter" => 500,"epsIter" => 1.e-3))

(Complex{Float64}[-1.197237252086016 + 0.4498363042506125im 0.6784383624116359 + 0.5660791048163021im -0.7849864638452732 - 0.01483593513452497im; 0.3788396401411271 + 0.2511463936422992im -0.974262084689121 - 0.2574674292040156im -0.6433234546249749 - 0.670531077035846im; -0.5027488452521038 + 0.616685953317809im 0.9073543082281365 + 0.35258183865185777im -0.0088033550999379 - 0.9139555194450166im], Dict{String,Any}("d*" => 2.8904022783502117,"d0" => 31.118041861573218,"nIter" => 27,"epsIter" => 0.001,"maxIter" => 500))

The reported error is the apolar norm between P and the approximated polynomial.

The initial error "d0" is ~ 31.11.

The algorithm rne_n_tr takes 10 iterations and decreases to the final error d* ~ 7.85, while the algorithm decreases to the final error ~ 2.89 after 27 iterations.